In [5]:
import os

import fsspec

from utils import get_logger
from utils.dask import create_cluster
from utils.hls.catalog import HLSCatalog
from utils.hls.catalog import HLSBand
from utils.hls.compute import calculate_job_median
from utils.hls.compute import jobs_from_catalog, process_jobs

In [2]:
os.environ['AZ_ACCESS_KEY_USFS']="g5Gj8mVdierdkMeQ7GpAWn+dDOabApqiSkzu8NQvIAgpxc7cyhB8spmzFbo/jomtG3A4nVJ2o1Pmn8CrIpMuDA=="
os.environ['AZ_ACCESS_KEY_LUMONITOREASTUS2']="]=9Pi0A+Cpea9GOhfpB/OSahr6mlygq/LgUS0yqQPBTPv8Dcooi0HDsUkHruH2c7N/vZumWceK+0hWVpkq+7MhYg=="
os.environ['AZURE_STORAGE_ACCOUNT'] = 'lumonitoreastus2'
os.environ['AZURE_STORAGE_ACCESS_KEY'] = os.environ['AZ_ACCESS_KEY_LUMONITOREASTUS2']

catalog_url = fsspec.get_mapper(
    f"az://fia/catalogs/hls_conus_2015-2019.zarr",
    account_name="usfs",
    account_key=os.environ['AZ_ACCESS_KEY_USFS']
)

catalog = HLSCatalog.from_zarr(catalog_url)
bands = [
    HLSBand.COASTAL_AEROSOL,
    HLSBand.BLUE,
    HLSBand.GREEN,
    HLSBand.RED,
    HLSBand.NIR_NARROW,
    HLSBand.SWIR1,
    HLSBand.SWIR2,
    HLSBand.QA  # needed for qa
]
 
catalog.xr_ds.attrs['bands'] = bands
# read the entire data once (each tile is 3660x3660)...
chunks = {'band': 1, 'x': 3660, 'y': 3660}

logger = get_logger('hls-conus-2016')

print(catalog.xr_ds)

<xarray.Dataset>
Dimensions:  (index: 599089)
Coordinates:
  * index    (index) int64 0 0 0 0 0 0 0 0 0 ... 986 986 986 986 986 986 986 986
Data variables:
    dt       (index) datetime64[ns] dask.array<chunksize=(74887,), meta=np.ndarray>
    scene    (index) object dask.array<chunksize=(74887,), meta=np.ndarray>
    sensor   (index) object dask.array<chunksize=(74887,), meta=np.ndarray>
    tile     (index) object dask.array<chunksize=(74887,), meta=np.ndarray>
    year     (index) int64 dask.array<chunksize=(74887,), meta=np.ndarray>
Attributes:
    bands:    [<HLSBand.COASTAL_AEROSOL: 1>, <HLSBand.BLUE: 2>, <HLSBand.GREE...


In [3]:
catalog.xr_ds = catalog.xr_ds.where(catalog.xr_ds['year']== 2016, drop=True)
# Had to do this as 2 steps b/c I was getting an error about duplicate indices
catalog.xr_ds = catalog.xr_ds.where(catalog.xr_ds['sensor']== 'L', drop=True)
catalog.xr_ds = catalog.xr_ds.where(catalog.xr_ds['tile']=='11ULP', drop=True)

print(catalog.xr_ds)

<xarray.Dataset>
Dimensions:  (index: 46)
Coordinates:
  * index    (index) int64 631 631 631 631 631 631 ... 631 631 631 631 631 631
Data variables:
    dt       (index) datetime64[ns] dask.array<chunksize=(46,), meta=np.ndarray>
    scene    (index) object dask.array<chunksize=(46,), meta=np.ndarray>
    sensor   (index) object dask.array<chunksize=(46,), meta=np.ndarray>
    tile     (index) object dask.array<chunksize=(46,), meta=np.ndarray>
    year     (index) float64 dask.array<chunksize=(46,), meta=np.ndarray>
Attributes:
    bands:    [<HLSBand.COASTAL_AEROSOL: 1>, <HLSBand.BLUE: 2>, <HLSBand.GREE...


In [7]:
jobs = jobs_from_catalog(catalog.xr_ds, 'tile')

cluster_args = dict(
    workers=64,
    worker_threads=1,
    worker_memory=1,
    scheduler_threads=4,
    scheduler_memory=8
)

process_jobs(
    jobs=jobs,
    job_fn=calculate_job_median,
    concurrency=2,  # because?
    checkpoint_path='./checkpoint',  # ?
    logger=logger,
    cluster_args=cluster_args,
    code_path='./utils',
    job_groupby='time.year',
    bands=bands,
    chunks=chunks,
    account_name='lumonitoreastus2',
    storage_container='hls',
    account_key=os.environ['AZ_ACCESS_KEY_LUMONITOREASTUS2']
)

2021-02-04 21:01:47,223 [INFO] hls-conus-2016 - Starting cluster
2021-02-04 21:01:56,339 [INFO] hls-conus-2016 - Cluster dashboard visible at /services/dask-gateway/clusters/default.a035a4ca4866483d9e5902ca8c493f89/status
2021-02-04 21:01:56,371 [INFO] hls-conus-2016 - Uploading code to cluster
2021-02-04 21:01:56,373 [INFO] hls-conus-2016 - Submitting job 11ULP
2021-02-04 21:02:42,851 [ERROR] hls-conus-2016 - Exception from dask cluster
Traceback (most recent call last):
  File "/home/jovyan/di-cog/utils/hls/compute.py", line 264, in run_job_subset
    result = future.result()
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/client.py", line 224, in result
    raise exc.with_traceback(tb)
  File "/home/jovyan/dask-worker-space/dask-worker-space/worker-3igmacow/source.zip/utils/hls/compute.py", line 196, in calculate_job_median
  File "/home/jovyan/dask-worker-space/dask-worker-space/worker-3igmacow/source.zip/utils/hls/compute.py", line 138, in save_to_zarr
  F

In [ ]:
cluster.shutdown()